In [201]:
# https://github.com/Datadolittle/Photo_Mosaic
import os, random, argparse
from PIL import Image
import numpy as np

In [202]:
def getImages(images_directory):
    files = os.listdir(images_directory)
    images = []
    for file in files:
        filePath = os.path.abspath(os.path.join(images_directory, file))
        try:
            fp = open(filePath, "rb")
            im = Image.open(fp)
            images.append(im)
            im.load()
            fp.close()
        except:
            print("Invalid image: %s" % (filePath,))
    return (images)


In [203]:
def getAverageRGB(image):
    image = image.convert('RGB')
    im = np.array(image)
    w, h, d = im.shape
    return (tuple(np.average(im.reshape(w * h, d), axis=0)))

In [204]:
def splitImage(image, size):
    W, H = image.size[0], image.size[1]
    m, n = size
    w, h = int(W / n), int(H / m)
    imgs = []
    for j in range(m):
        for i in range(n):
            imgs.append(image.crop((i * w, j * h, (i + 1) * w, (j + 1) * h)))
    return (imgs)

In [205]:
def getBestMatchIndex(input_avg, avgs):
    avg = input_avg
    index = 0
    min_index = 0
    min_dist = float("inf")
    for val in avgs:
        dist = ((val[0] - avg[0]) * (val[0] - avg[0]) +
                (val[1] - avg[1]) * (val[1] - avg[1]) +
                (val[2] - avg[2]) * (val[2] - avg[2]))
        if dist < min_dist and index not in MATCH_INDECES:
            min_dist = dist
            min_index = index
        index += 1
    # Global store of matched indexes if no reuse
    if not reuse_images:
        MATCH_INDECES.append(min_index)

    return (min_index)


In [206]:
def createImageGrid(images, dims):
    m, n = dims
    width = max([img.size[0] for img in images])
    height = max([img.size[1] for img in images])
    grid_img = Image.new('RGB', (n * width, m * height))
    for index in range(len(images)):
        row = int(index / n)
        col = index - n * row
        grid_img.paste(images[index], (col * width, row * height))
    return (grid_img)


In [239]:
def createPhotomosaic(target_image, input_images, grid_size,
                      reuse_images):
    target_images = splitImage(target_image, grid_size)

    output_images = []
    count = 0
    batch_size = int(len(target_images) / 10)
    avgs = []
    for img in input_images:
        try:
            avgs.append(getAverageRGB(img))
        except ValueError:
            continue

    match_index_list = []
    for img in target_images:
        avg = getAverageRGB(img)
        match_index = getBestMatchIndex(avg, avgs)
        output_images.append(input_images[match_index])
        match_index_list.append(match_index)
        if count > 0 and batch_size > 10 and count % batch_size == 0:
            print('processed %d of %d...' % (count, len(target_images)))
        count += 1

    mosaic_image = createImageGrid(output_images, grid_size)
    return (mosaic_image, match_index_list)

In [208]:
from types import SimpleNamespace

In [246]:
args_dic = {}

In [247]:
#python Mosaic_Creator.py --target Mona_Lisa.jpg --images img/ --grid 100 100 --output Mona_Lisa_Mosaic.jpeg
#args_dic['target'] = 'Mona_Lisa_small.jpg'
args_dic['target'] = 'Mona_Lisa.jpg'
args_dic['images'] = 'img/'
args_dic['grid'] = (83,185)
args_dic['output'] = 'Mona_Lisa_Mosaic.jpeg'
args_dic['reuse'] = True
args_dic['resize'] = True

In [248]:
args = SimpleNamespace(**args_dic)

In [249]:
MATCH_INDECES = []

In [250]:
target_image = Image.open(args.target)

In [251]:
target_image.size

(7479, 11146)

In [252]:
# input images
print('reading input folder...')
input_images = getImages(args.images)

reading input folder...
Invalid image: /Volumes/Transcend/GitHub/python-photo-mosaic/img/product


In [253]:
args.images

'img/'

In [254]:
import glob

In [255]:
tile_path_list = glob.glob("img/**/*.png", recursive=True)

In [256]:
input_images = []
for tile in tile_path_list:
    try:
        im = Image.open(tile)
    except:
        continue
    input_images.append(im)

In [257]:
input_images[0].size

(90, 60)

In [258]:
target_image.width // input_images[0].width

83

In [259]:
target_image.height // input_images[0].height

185

In [260]:
# size of grid
grid_size = (int(args.grid[1]), int(args.grid[0]))

In [261]:
grid_size

(185, 83)

In [262]:
output_filename = args.output

In [263]:
# re-use any image in input
reuse_images = args.reuse

In [264]:
# resize the input to fit original image size?
resize_input = args.resize

In [265]:
print('starting photomosaic creation...')

starting photomosaic creation...


In [266]:
target_image.size[0] / grid_size[1]

90.10843373493977

In [267]:
target_image.size[1] / grid_size[0]

60.24864864864865

In [268]:
# resizing input
if resize_input:
    print('resizing images...')
    # for given grid size, compute max dims w,h of tiles
    dims = (int(target_image.size[0] / grid_size[1]),
            int(target_image.size[1] / grid_size[0]))
    print("max tile dims: %s" % (dims,))
    # resize
    for img in input_images:
        img.thumbnail(dims)

resizing images...
max tile dims: (90, 60)


In [269]:
# create photomosaic
mosaic_image, match_index_list = createPhotomosaic(target_image, input_images, grid_size, reuse_images)

processed 1535 of 15355...
processed 3070 of 15355...
processed 4605 of 15355...
processed 6140 of 15355...
processed 7675 of 15355...
processed 9210 of 15355...
processed 10745 of 15355...
processed 12280 of 15355...
processed 13815 of 15355...
processed 15350 of 15355...


In [270]:
# write out mosaic
mosaic_image.save(output_filename, 'jpeg')

In [271]:
len(match_index_list)

15355

In [272]:
from collections import Counter

In [273]:
c = Counter(match_index_list)

In [275]:
top10 = c.most_common(10)

In [276]:
top10

[(69, 1685),
 (63, 1663),
 (59, 974),
 (85, 943),
 (210, 825),
 (176, 784),
 (208, 668),
 (248, 605),
 (179, 469),
 (206, 393)]

In [277]:
tile_path_list[69]

'img/product/capacitor/ceramic/ceralink/010107_ceralink_b58031u5_pi0101.png'

In [278]:
os.path.basename(tile_path_list[69])

'010107_ceralink_b58031u5_pi0101.png'

In [286]:
total = len(match_index_list)
for i, n in top10:
    #print(tile_path_list[i])
    basename = os.path.basename(tile_path_list[i])
    category = tile_path_list[i].split('/')[-2]
    name = basename.split('.')[0]
    share = (n / total) * 100
    print("{:.1f}%: {}, {}".format(share, category, name))

11.0%: ceralink, 010107_ceralink_b58031u5_pi0101
10.8%: uhv, 010105_tsf-40c_pi0801s
6.3%: mlcc, 010101_cna5l-b_pi0101
6.1%: line-filter, 030006_aml0519_pi0101
5.4%: piezolisten, 080401_phua3030-049b_top_pi0101
5.1%: ptc-thermistor, 050203_c810-c995_tpt0521-v_pi0101
4.4%: piezohapt, 080101_phua3015-30a-21-000_fpc_pi0101
3.9%: tx-coil-module, 160008_wt505090-10f2-a11-g1_pi0101
3.1%: ptc-thermistor, 050203_c1412_c1451_tpt1075-t_pi0101
2.6%: ptc_switch, 080002_j286_j29_tpt05585_pi0101
